## SAT179_2_iNaturalist

* [#179](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/179) "Koppla SAT till artobservationer"
* notebooks
    * [SAT179_iNaturalist.ipynb](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/SAT179_iNaturalist.ipynb)
    * [SAT179_2_iNaturalist.ipynb](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/SAT179_2_iNaturalist.ipynb)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-23 15:23


In [2]:
from string import Template
from datetime import datetime as dt
import html  # vi använder html.escape här

def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
    offset_px=(10, 54),  # (top, left) – 54px för att hamna snällt bredvid zoom
):
    """Superenkel, robust About-box som alltid syns."""
    if created_date is None:
        created_date = dt.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"
    top, left  = offset_px
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {html.escape(t)}</a></div>'
        for t, u in links
    )

    tpl = Template(r"""
<style>
  .sat-about {
    position: fixed; z-index: 10000;
    top: ${top}px; left: ${left}px;
    background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    min-width: 240px; max-width: 320px; pointer-events: auto;
  }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>

<div id="${box_id}" class="sat-about ${collapsed_class}">
  <div id="${header_id}" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="${issue_url}" target="_blank">#${issue_number}</a>&nbsp;&nbsp; Map: ${map_name}</div>
    <div>Created: ${created_date}</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">${links_html}</div>
  </div>
</div>

<script>
(function(){
  var boxId = "${box_id}";
  var hdrId = "${header_id}";
  var storageKey = "satAboutCollapsed_${map_dom_id}_#${issue_number}";

  function setCollapsed(box, collapsed) {
    if (!box) return;
    if (collapsed) box.classList.add("sat-about-collapsed");
    else box.classList.remove("sat-about-collapsed");
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function init(){
    var box = document.getElementById(boxId);
    var hdr = document.getElementById(hdrId);
    if (!box || !hdr) return;

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      setCollapsed(box, !box.classList.contains("sat-about-collapsed"));
    });
  }

  if (document.readyState === "loading") {
    document.addEventListener("DOMContentLoaded", init);
  } else {
    init();
  }
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=html.escape(map_name),
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
        top=top, left=left,
    )
    m.get_root().html.add_child(folium.Element(html_code))


In [3]:
import json, time, requests, folium
from shapely.geometry import shape, Polygon
from shapely.ops import unary_union

INAT_URL = "https://api.inaturalist.org/v1/observations" 

def _get_with_backoff(url, params, timeout=30, tries=5, base_wait=2):
    """GET with exponential backoff + jitter."""
    attempt = 0
    while True:
        r = requests.get(url, params=params, timeout=timeout) 
        
        if r.status_code in (429, 500, 502, 503, 504):
            attempt += 1
            if attempt >= tries:
                r.raise_for_status()
            wait = (base_wait ** attempt) + random.uniform(0, 1)
            print("/t Wait: ",str(wait))
            time.sleep(wait)
            continue
        r.raise_for_status()
        return r


def fetch_inat_all_parts(geom_like,
                         iconic_taxa=None, d1=None, d2=None,
                         per_page=200, max_pages=5, pause=0.2,
                         show_progress=True, desc="Fetch",
                         # --- NYTT: valfri extern bbox ---
                         swlat=None, swlng=None, nelat=None, nelng=None):
    """
    Tar LineString/Polygon/MultiPolygon, buffrar vid behov, loopar delpolygoner,
    deduplar observationer och visar progressbar.

    NYTT: Du kan skicka in en extern bbox (swlat, swlng, nelat, nelng).
    Den läggs in i grundparametrarna och används även i bbox-fallbacken.
    """
    g = geom_like if hasattr(geom_like, "geom_type") else shape(geom_like)
    if g.geom_type == "LineString":
        g = g.buffer(0.02)  # ~200m
    elif g.geom_type not in ("Polygon", "MultiPolygon"):
        g = unary_union(g)

    polys = [g] if isinstance(g, Polygon) else [p for p in g.geoms if p.area > 1e-10]

    base = {}
    if iconic_taxa: base["iconic_taxa"] = iconic_taxa
    if d1: base["d1"] = d1
    if d2: base["d2"] = d2
    base["geoprivacy"] = "open"    # <<< add this

    # --- NYTT: om extern bbox medföljer, lägg till den i base (snävare sökningar)
    if all(v is not None for v in (swlat, swlng, nelat, nelng)):
        base.update({"swlat": swlat, "swlng": swlng, "nelat": nelat, "nelng": nelng})

    total_steps = len(polys) * max_pages  # ungefärligt (per polygon x sidor)
    pbar = tqdm(total=total_steps, desc=desc, unit="page", leave=False) if show_progress else None

    all_obs = {}
    try:
        for poly in polys:
            part = _inat_query_for_polygon(
                poly, base,
                per_page=per_page, max_pages=max_pages, pause=pause, pbar=pbar,
                # --- NYTT: skicka med extern bbox till funktionen
                ext_bbox=(swlat, swlng, nelat, nelng) if all(v is not None for v in (swlat, swlng, nelat, nelng)) else None
            )
            for o in part:
                all_obs[o["id"]] = o
    finally:
        if pbar: pbar.close()
    return list(all_obs.values())

def _polygon_param_from_poly(poly: Polygon, simplify_tol: float = 0.0002, step: int = 1):
    p = poly.simplify(simplify_tol, preserve_topology=True)
    ring = list(p.exterior.coords)[::step]
    if ring[0] != ring[-1]:
        ring.append(ring[0])
    return ",".join(f"{lng},{lat}" for (lng, lat) in ring)

def _bbox_params_from_poly(poly: Polygon):
    minx, miny, maxx, maxy = poly.bounds
    return {"swlng": minx, "swlat": miny, "nelng": maxx, "nelat": maxy}

def _inat_query_for_polygon(poly: Polygon, params_base: dict,
                            per_page=200, max_pages=5, pause=0.2, pbar=None,
                            # --- NYTT: extern bbox vidarebefordras hit
                            ext_bbox=None):
    """
    Kör iNat mot polygon. Har adaptiv förenkling + bbox-fallback.
    Om ext_bbox ges (swlat, swlng, nelat, nelng) används den som global
    insnävning och i fallbacken.
    """
    
    results = []
    params = dict(params_base)
    params.update({
        "verifiable": "true",
        "order_by": "observed_on",
        "order": "desc",
        "per_page": per_page
    })

    # 1) Adaptiv polygon (för att undvika 414). Om ext_bbox finns skickas den med
    #    samtidigt — iNat accepterar bbox + polygon och tolkar det snävare.
    for simplify_tol in (0.0002, 0.0004, 0.0008, 0.0016):
        for step in (1, 2, 3, 4):
            polygon_param = _polygon_param_from_poly(poly, simplify_tol=simplify_tol, step=step)
            params_poly = dict(params, polygon=polygon_param)
            try:
                # Första sidan
                r = _get_with_backoff(INAT_URL, dict(params_poly, page=1), timeout=30)
                if r.status_code == 414:
                    raise requests.HTTPError("414 URI Too Large (polygon)", response=r)
                r.raise_for_status()
                data = r.json()
                batch = data.get("results", [])
                results.extend(batch)
                if pbar: pbar.update(1)

                # Paginerat
                for page in range(2, max_pages + 1):
                    r = _get_with_backoff(INAT_URL, dict(params_poly, page=page), timeout=30)
                    r.raise_for_status()
                    b2 = r.json().get("results", [])
                    results.extend(b2)
                    if pbar: pbar.update(1)
                    if len(b2) < per_page:
                        break
                    time.sleep(pause)
                return results
            except requests.HTTPError as e:
                # prova nästa simplifiering—bubbla inte upp 414
                if getattr(e, "response", None) is None or e.response.status_code != 414:
                    raise

    # 2) Fallback till bbox
    #    Om ext_bbox finns -> använd den. Annars -> bbox från polygonen.
    if ext_bbox is not None:
        swlat, swlng, nelat, nelng = ext_bbox
        params_bbox = dict(params, swlat=swlat, swlng=swlng, nelat=nelat, nelng=nelng)
    else:
        params_bbox = dict(params, **_bbox_params_from_poly(poly))

    for page in range(1, max_pages + 1):
        r = _get_with_backoff(INAT_URL, dict(params_bbox, page=page), timeout=30)
        r.raise_for_status()
        b = r.json().get("results", [])
        results.extend(b)
        if pbar: pbar.update(1)
        if len(b) < per_page:
            break
        time.sleep(pause)
    return results


In [4]:
def obs_to_feature(o, color="#3388ff"):
    gj = o.get("geojson") or {}
    if not (isinstance(gj.get("coordinates"), (list, tuple)) and len(gj["coordinates"]) >= 2):
        return None
    lon, lat = gj["coordinates"][:2]

    # Prefer full ISO time, else fall back to date
    obs_time = o.get("time_observed_at") or o.get("observed_on") or o.get("created_at")
    if not obs_time:   # no timestamp → skip
        return None
    # normalize to date if it's longer than YYYY-MM-DD
    if len(obs_time) > 10:
        obs_time = obs_time[:10]

    taxon = o.get("taxon") or {}
    cname = taxon.get("preferred_common_name") or taxon.get("name") or "okänd"
    sname = taxon.get("name") or ""
    oid   = o.get("id", "")
    url   = f"https://www.inaturalist.org/observations/{oid}"
    api   = f"https://api.inaturalist.org/v1/observations/{oid}"

    img = ""
    photos = o.get("photos") or []
    if photos:
        raw = photos[0].get("url", "")
        if raw:
            img = raw.replace("square", "medium")

    popup = f"<b>{cname}</b><br><i>{sname}</i><br>{obs_time}<br>"
    if img:
        popup += f"<img src='{img}' width='220'><br>"
    popup += f"<a href='{url}' target='_blank'>Visa i iNaturalist</a> · "
    popup += f"<a href='{api}' target='_blank'>API JSON</a>"

    return {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lon, lat]},
        "properties": {
            "time": obs_time,          # ← the slider reads this
            "popup": popup,
            "icon": "circle",
            "iconstyle": {"color": color, "fillColor": color, "fillOpacity": 0.8}
        }
    }


In [6]:
import json
from datetime import date, timedelta
from pathlib import Path

from shapely.geometry import mapping, shape, LineString, MultiLineString, Polygon, MultiPolygon, Point
from shapely.ops import unary_union

import folium
from folium.plugins import TimestampedGeoJson

# ---- Progressbar (fallback) ----
try:
    from tqdm.auto import tqdm
except ImportError:
    def tqdm(x, **kwargs):
        return x

# ============ 0) Läs leden ============
TRAIL_FILE = Path("SAT_full.geojson")
if not TRAIL_FILE.exists():
    raise FileNotFoundError("Hittar inte SAT_full.geojson.")

with TRAIL_FILE.open(encoding="utf-8") as f:
    gj = json.load(f)

lines = []
for feat in gj.get("features", []):
    geom = shape(feat.get("geometry"))
    if isinstance(geom, LineString):
        if len(geom.coords) >= 2:
            lines.append(geom)
    elif isinstance(geom, MultiLineString):
        for seg in geom.geoms:
            if len(seg.coords) >= 2:
                lines.append(seg)

if not lines:
    raise ValueError("SAT_full.geojson innehåller inga linjesegment.")

trail = unary_union(lines)
center_lat, center_lon = trail.centroid.y, trail.centroid.x


# ============ 1) Buffert (≈2000 m) + bbox ============
# Vid ~59–60°N motsvarar ~0.02° ungefär 2000 m
BUFFER_DEG = 0.02
search_geom = trail.buffer(BUFFER_DEG)
if isinstance(search_geom, (Polygon, MultiPolygon)):
    search_geom = search_geom.simplify(0.0002, preserve_topology=True)


minx, miny, maxx, maxy = search_geom.bounds   # (lon_min, lat_min, lon_max, lat_max)
swlng, swlat, nelng, nelat = minx, miny, maxx, maxy

# ============ 2) Datumintervall ============
# just yesterday → today
d2 = date.today().isoformat()
#d1 = (date.today() - timedelta(days=4000)).isoformat()
d1 = (date.today() - timedelta(days=10)).isoformat()
print(d1,d2)
# ============ 3) Färger ============
TAXON_COLORS = {
    "Plantae":   "#2b8a3e",
    "Aves":      "#1d4ed8",
    "Mammalia":  "#78350f",
    "Reptilia":  "#059669",
    "Amphibia":  "#14b8a6",
    "Insecta":   "#eab308",
    "Arachnida": "#7f1d1d",
    "Mollusca":  "#9333ea",
    "Animalia":  "#6b7280",
}

# ============ 4) Helpers ============
def obs_in_geom(o, geom) -> bool:
    gj = o.get("geojson")
    if not (isinstance(gj, dict) and isinstance(gj.get("coordinates"), (list, tuple)) and len(gj["coordinates"]) >= 2):
        return False
    lon, lat = gj["coordinates"][:2]
    try:
        return geom.contains(Point(lon, lat)) or geom.touches(Point(lon, lat))
    except Exception:
        return False

def obs_to_feature(o, color="#3388ff"):
    gj = o.get("geojson") or {}
    if not (isinstance(gj.get("coordinates"), (list, tuple)) and len(gj["coordinates"]) >= 2):
        return None
    lon, lat = gj["coordinates"][:2]

    # Prefer full ISO time, else fall back to date
    obs_time = o.get("time_observed_at") or o.get("observed_on") or o.get("created_at")
    if not obs_time:   # no timestamp → skip
        return None
    # normalize to date if it's longer than YYYY-MM-DD
    if len(obs_time) > 10:
        obs_time = obs_time[:10]

    taxon = o.get("taxon") or {}
    cname = taxon.get("preferred_common_name") or taxon.get("name") or "okänd"
    sname = taxon.get("name") or ""
    oid   = o.get("id", "")
    url   = f"https://www.inaturalist.org/observations/{oid}"
    api   = f"https://api.inaturalist.org/v1/observations/{oid}"

    img = ""
    photos = o.get("photos") or []
    if photos:
        raw = photos[0].get("url", "")
        if raw:
            img = raw.replace("square", "medium")

    popup = f"<b>{cname}</b><br><i>{sname}</i><br>{obs_time}<br>"
    if img:
        popup += f"<img src='{img}' width='220'><br>"
    popup += f"<a href='{url}' target='_blank'>Visa i iNaturalist</a> · "
    popup += f"<a href='{api}' target='_blank'>API JSON</a>"

    return {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lon, lat]},
        "properties": {
            "time": obs_time,          # ← the slider reads this
            "popup": popup,
            "icon": "circle",
            "iconstyle": {"color": color, "fillColor": color, "fillOpacity": 0.8}
        }
    }

def to_fc(obs, color="#3388ff", desc=None, radius=4, fill_opacity=0.6):
    """
    Bygger ett FeatureCollection av observationer.
    radius: pixelradie för punkter (CircleMarker m.m.)
    fill_opacity: fyllnadsopacitet för punkter
    """
    feats, bounds, skipped = [], [], 0
    for o in tqdm(obs, desc=desc, leave=False):
        if not obs_in_geom(o, search_geom):   # hårdklipp till 2000 m från leden
            skipped += 1
            continue

        # Om din obs_to_feature kan ta emot extra stil-parametrar, låt dem gå in här:
        try:
            f = obs_to_feature(o, color=color, radius=radius, fill_opacity=fill_opacity)
        except TypeError:
            # Backoff om obs_to_feature inte har dessa argument
            f = obs_to_feature(o, color)

        if f:
            # Säkerställ att radius m.m. finns i properties
            props = f.setdefault("properties", {})
            props.setdefault("radius", radius)
            style = props.setdefault("style", {})
            style.setdefault("radius", radius)
            style.setdefault("color", color)
            style.setdefault("fillColor", color)
            style.setdefault("fillOpacity", fill_opacity)

            feats.append(f)
            lon, lat = f["geometry"]["coordinates"]
            bounds.append((lat, lon))
        else:
            skipped += 1

    return {"type": "FeatureCollection", "features": feats}, bounds, skipped

        
# ============ 5) Hämta observationer inom bbox (och klipp till buffert) ============
animal_taxa = ["Aves","Mammalia","Reptilia","Amphibia","Insecta","Arachnida","Mollusca","Animalia"]


taxa_groups = [
    ("Aves", "Birds"),
    ("Plantae", "Plants"),
    ("Mammalia", "Mammals"),
    ("Fungi", "Fungi"),
    ("Insecta", "Insects"),
    ("Amphibia", "Amphibians"),
    ("Reptilia", "Reptiles")
]

all_groups = {}

for iconic_taxa, desc in taxa_groups:
    all_groups[iconic_taxa] = fetch_inat_all_parts(
        search_geom,
        iconic_taxa=iconic_taxa,
        d1=d1, d2=d2,
        per_page=200, max_pages=5,
        show_progress=True,
        desc=desc,
        swlat=swlat, swlng=swlng,
        nelat=nelat, nelng=nelng
    )


# ============ 6) Bygg Folium-kartan ============
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, control_scale=True)

# Leden
folium.GeoJson(
    mapping(trail),
    name="SAT-led",
    style_function=lambda x: {"color": "#1e40af", "weight": 3}
).add_to(m)

# Bufferten (sökzonen)
folium.GeoJson(
    mapping(search_geom),
    name="Sökzon (2000 m)",
    style_function=lambda x: {"color": "#3388ff", "weight": 1, "fillOpacity": 0.05}
).add_to(m)

# --- Samla alla features i ett enda tidslager ---
all_bounds = []
all_features = []
total_skipped = 0

for tx, obs in tqdm(all_groups.items(), desc="Bygger features", total=len(all_groups)):
    color = TAXON_COLORS.get(tx, "#3388ff")
#    fc, bounds, skipped = to_fc(obs, color=color, desc=f"{tx} → features")
    fc, bounds, skipped = to_fc(obs, color=color, desc=f"{tx} → features", radius=4)
    all_features.extend(fc["features"])
    all_bounds.extend(bounds)
    total_skipped += skipped

# Bygg en enda FeatureCollection
fc_all = {"type": "FeatureCollection", "features": all_features}

# after building fc_all:
n = len(fc_all["features"])
print(f"Features in time layer: {n}")
if not n:
    print("No features → widen date range, loosen buffer/bbox, or fetch one taxon with per_page=50, max_pages=1.")


# Lägg till ett gemensamt tidslager 
days_total = 3650   # ungefär 10 år
total_ms = 10000    # 10 sek total
transition_time = int(total_ms / days_total)  # ≈ 3 ms

# 1) Timeline-lagret
TimestampedGeoJson(
    fc_all,
    period="P1D",        # ett steg = 1 dag
    duration="P1Y",      # punkten syns 1 år
    auto_play=True,
    loop=True,
    transition_time=transition_time
).add_to(m) 


# 2) Statiskt lager (alla punkter) — with popups + circle styling
static_layer = folium.GeoJson(
    fc_all,
    name="Alla observationer",
    point_to_layer=lambda feature, latlng: folium.CircleMarker(
        location=latlng,
        radius=feature["properties"].get("radius", 3),
        color=feature["properties"].get("style", {}).get("color", "black"),
        fill=True,
        fill_color=feature["properties"].get("style", {}).get("fillColor", "black"),
        fill_opacity=feature["properties"].get("style", {}).get("fillOpacity", 0.4),
        weight=1,
    ),
    popup=folium.GeoJsonPopup(
        fields=["popup"],
        labels=False,
        parse_html=True,
        max_width=350
    ),
    show=True  # ensure layer is visible by default
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

if total_skipped:
    tqdm.write(f"⚠️ Hoppade över totalt {total_skipped} obs (utanför buffert/utan data).")

# --- Liten legend med antal per taxa ---
from branca.element import MacroElement
from jinja2 import Template

# Räkna features per taxa (efter filtrering)
taxon_counts = []
for tx, obs in all_groups.items():
    color = TAXON_COLORS.get(tx, "#888")
    fc, _, _ = to_fc(obs, color=color, desc=f"{tx} → features", radius=4)  # använder samma filter (2000 m)
    taxon_counts.append((tx, color, len(fc["features"])))



legend_html = """
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 150px; 
            background: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px;">
<b>Taxa</b><br>
<i style="color:#2b8a3e;">●</i> Växter<br>
<i style="color:#1d4ed8;">●</i> Fåglar<br>
<i style="color:#78350f;">●</i> Däggdjur<br>
<i style="color:#eab308;">●</i> Insekter<br>
<i style="color:#14b8a6;">●</i> Groddjur<br>
<i style="color:#059669;">●</i> Kräldjur<br>
<i style="color:#7f1d1d;">●</i> Spindeldjur<br>
<i style="color:#9333ea;">●</i> Blötdjur<br></div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

    
from branca.element import MacroElement
from jinja2 import Template



# Auto-zoom om vi har punkter
if all_bounds:
    m.fit_bounds(all_bounds)




2025-09-13 2025-09-23


Birds:   0%|          | 0/75 [00:00<?, ?page/s]

Plants:   0%|          | 0/75 [00:00<?, ?page/s]

Mammals:   0%|          | 0/75 [00:00<?, ?page/s]

Fungi:   0%|          | 0/75 [00:00<?, ?page/s]

NameError: name 'random' is not defined

In [ ]:
import json
from collections.abc import Mapping

def json_sanitize(o):
    if callable(o):
        return None                     # drop functions
    if o is None or isinstance(o, (str, int, float, bool)):
        return o
    if isinstance(o, list):
        return [json_sanitize(v) for v in o]
    if isinstance(o, Mapping):
        return {str(k): json_sanitize(v) for k, v in o.items()}
    # try ISO for datetime-like
    if hasattr(o, "isoformat"):
        try:
            return o.isoformat()
        except Exception:
            pass
    # last resort
    return str(o)

def assert_jsonable(x, label="object"):
    try:
        json.dumps(x)
    except TypeError as e:
        raise TypeError(f"{label} is not JSON-serializable: {e}")

In [ ]:
def first_bad(obj, path="root"):
    try:
        json.dumps(obj); return None
    except TypeError:
        pass
    if isinstance(obj, dict):
        for k, v in obj.items():
            bad = first_bad(v, f"{path}.{k}")
            if bad: return bad
    elif isinstance(obj, list):
        for i, v in enumerate(obj):
            bad = first_bad(v, f"{path}[{i}]")
            if bad: return bad
    return (path, type(obj).__name__)

bad = first_bad(fc)
if bad: print("Non-serializable at:", bad)

In [ ]:
import json
from collections.abc import Mapping

def json_sanitize(o):
    """Convert object into something JSON can handle."""
    if callable(o):
        return None  # remove functions
    if o is None or isinstance(o, (str, int, float, bool)):
        return o
    if isinstance(o, list):
        return [json_sanitize(v) for v in o]
    if isinstance(o, Mapping):
        return {str(k): json_sanitize(v) for k, v in o.items()}
    if hasattr(o, "isoformat"):  # datetime-like
        try:
            return o.isoformat()
        except Exception:
            pass
    return str(o)  # fallback


In [ ]:
import json
from collections.abc import Mapping, Sequence

def find_first_callable(obj, path="root"):
    """Locate first function/callable inside a nested dict/list structure."""
    if callable(obj):
        return path
    if isinstance(obj, Mapping):
        for k, v in obj.items():
            where = find_first_callable(v, f"{path}.{k}")
            if where: return where
    elif isinstance(obj, (list, tuple)):
        for i, v in enumerate(obj):
            where = find_first_callable(v, f"{path}[{i}]")
            if where: return where
    return None

def _json_default(o):
    """How to serialize odd objects when round-tripping through JSON."""
    if callable(o):
        return None  # kill functions
    # shapely / geo-like
    if hasattr(o, "__geo_interface__"):
        return o.__geo_interface__
    # datetime-like
    if hasattr(o, "isoformat"):
        try:
            return o.isoformat()
        except Exception:
            pass
    # numpy & friends or anything else
    try:
        return str(o)
    except Exception:
        return None

def make_json_safe(obj):
    """
    Force object into JSON-serializable form by round-tripping via json.
    Any non-serializable items (incl. functions) are converted using _json_default.
    """
    return json.loads(json.dumps(obj, default=_json_default))


In [ ]:
for tx, obs in tqdm(all_groups.items(), desc="Bygger features", total=len(all_groups)):
    color = TAXON_COLORS.get(tx, "#3388ff")
    fc, bounds, skipped = to_fc(obs, color=color, desc=f"{tx} → features")

    # 1) (Optional) Tell me exactly where the first function is
    bad_path = find_first_callable(fc)
    if bad_path:
        print(f"⚠️ {tx}: callable found at {bad_path} — cleaning it now.")

    # 2) Force the FeatureCollection to be JSON-safe (removes functions)
    safe_fc = make_json_safe(fc)

    # 3) Double-check we’re clean; if not, skip this taxon so save() won’t crash
    still_bad = find_first_callable(safe_fc)
    if still_bad:
        print(f"⏭️  Skipping {tx}: still has a callable at {still_bad} after cleaning.")
        continue

    n = len(safe_fc.get("features", []))
    #taxon_counts[tx] = n
    total_skipped += skipped

    # Build a layer for this taxon
    fg = folium.FeatureGroup(name=f"{tx} ({n})", show=True)
    folium.GeoJson(
        safe_fc,
        name=tx,
        # keep functions ONLY as arguments here (that’s fine — Folium won’t serialize them in GeoJSON)
        style_function=lambda _x, c=color: {"color": c, "weight": 2, "fillOpacity": 0.3},
        tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["Namn"], labels=True, sticky=False),
    ).add_to(fg)
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
add_about_box(m, issue_number=179, map_name="SAT iNaturalist")

OUT = "output/179_inat_taxa_layers_colored.html"
m.save(OUT)
print(f"✅ Sparade: {OUT}")


In [ ]:
# --- Samla alla features + räkna per taxa ---
all_bounds = []
all_features = []
taxon_counts = {}
total_skipped = 0

for tx, obs in tqdm(all_groups.items(), desc="Bygger features", total=len(all_groups)):
    color = TAXON_COLORS.get(tx, "#3388ff")
    fc, bounds, skipped = to_fc(obs, color=color, desc=f"{tx} → features")
    n = len(fc["features"])
    taxon_counts[tx] = n
    all_features.extend(fc["features"])
    all_bounds.extend(bounds)
    total_skipped += skipped

# Summering
print("Antal observationer per taxa (inom buffert):")
for tx, n in taxon_counts.items():
    print(f"  {tx}: {n}")
print(f"Totalt: {sum(taxon_counts.values())}")
print(f"Hoppade över totalt {total_skipped}")


In [ ]:
end_time = time.time()
duration = end_time - start_time
print(f"GeoJSON saved to {OUT}")
print(f"Finished in {duration:.2f} seconds.")


In [ ]:
# Arholma→Landsort (grovt exempel – justera om du vill)
ARHOLMA_LANDSORT_BBOX = {
    "swlat": 58.6,  "swlng": 17.3,   # sydväst (lat, lon)
    "nelat": 60.3,  "nelng": 19.6,   # nordost (lat, lon)
}
# ... och skicka dessa värden i dina fetch_inat_all_parts(...) anrop
